In [1]:
import pandas as pd
import scipy.signal as signal
import numpy as np

In [2]:
# -------------------------
# SensorID to Body Part Map
# -------------------------
SENSOR_MAP = {
    1: "LT",
    2: "RT",
    3: "LS",
    4: "RS",
    5: "LA",
    6: "RA"
}

In [3]:
# -------------------------
# Load IMU data
# -------------------------
imu_df = pd.read_csv("dataset with force/Test_13-3-2025/Specimen_1/IMU/test1.csv")  # path to your combined IMU file
imu_df = imu_df.sort_values("Time(ms)").reset_index(drop=True)


In [4]:
# Pivot to wide format for each body part
imu_wide = {}
for sensor_id, label in SENSOR_MAP.items():
    df_part = imu_df[imu_df["SensorID"] == sensor_id].copy()
    df_part = df_part.rename(columns={
        "AccelX": f"{label}_AccelX",
        "AccelY": f"{label}_AccelY",
        "AccelZ": f"{label}_AccelZ",
        "GyroX": f"{label}_GyroX",
        "GyroY": f"{label}_GyroY",
        "GyroZ": f"{label}_GyroZ",
        "Time(ms)": "Time"
    })
    df_part = df_part.drop(columns=["SensorID"])
    imu_wide[label] = df_part

In [5]:
# Merge all body part IMU data on time
combined_imu = imu_wide["RS"]
for part in ["RA", "RT", "LT", "LA", "LS"]:
    combined_imu = pd.merge_asof(combined_imu, imu_wide[part], on="Time", direction="nearest")


In [6]:
# -------------------------
# Load Force Sensor Data
# -------------------------
left_force = pd.read_csv("dataset with force/Test_13-3-2025/Specimen_1/Left_Force/test1.csv")   # path to left force sensor
right_force = pd.read_csv("dataset with force/Test_13-3-2025/Specimen_1/Right_Force/test1.csv") # path to right force sensor


In [7]:
# Resample to match combined_imu length
def resample_force(force_df, target_len):
    force = force_df.sort_values("Time(ms)")["ForceValue"].values
    return signal.resample(force, target_len)

combined_imu["Left_Force"] = resample_force(left_force, len(combined_imu))
combined_imu["Right_Force"] = resample_force(right_force, len(combined_imu))

In [8]:
# -------------------------
# Final Output
# -------------------------
combined_imu.to_csv("combined_dataset.csv", index=False)
print("Saved combined_dataset.csv")


Saved combined_dataset.csv
